In [1]:
import numpy as np
import pandas as pd
import sys
import re
from scipy import sparse
from tabulate import tabulate
import matplotlib as mpl
import matplotlib.pyplot as plt

### Read SNAP data into list of lists

In [2]:
fname = './data/amazon0505.txt'
with open(fname, 'r') as f:
    lines = [[int(node) for node in re.split('\t',edge.strip('\n'))[:2]] for edge in f.readlines() if edge[0][0] != '#']
edges = np.array(lines)
numItems = len(np.unique(edges))

### Read small datasets into list of lists

In [3]:
# # fname = './data/karate.csv'
# fname = './data/lesmis.csv'
# # fname = './data/eg.txt'
# df = pd.read_csv(fname, header=None, usecols=[i for i in range(4)])
# if type(df[2][0]) == str:
#     df[2] = df[2].str.replace('"', '').str.strip()

# names = sorted(np.unique(np.concatenate((df[0].unique(),df[2].unique()))))

# a = np.array(df[0].apply(names.index))   
# b = np.array(df[2].apply(names.index))
# edges = np.array([b,a]).T
# numItems = len(names)

In [4]:
print(sys.getsizeof(edges))

53709304


### Create adjacency matrix 

In [5]:
adj = sparse.lil_matrix((np.max(edges)+1, np.max(edges)+1))

In [6]:
adj[edges[:,0], edges[:,1]] = 1

### Connect sink nodes to themselves

In [7]:
degOut = adj.getnnz(axis = 1) # num of non zero values in row

adj.setdiag(degOut == 0) # more efficient with lilmatrix
adj = adj.tocsr()
degOut = adj.getnnz(axis = 1) # num of non zero values in row

### Scale matrix by outgoing edges

In [8]:
degOutRep = np.repeat(degOut, degOut) # degOut is the same as number of data points in row
adj.data = adj.data / degOutRep

### Iterate pagerank

In [9]:
p = [1/adj.shape[0]] * adj.shape[0] 
d = 0.9
jProb = [(1-d)/len(p)] * len(p)
maxdiff = 1
numiters = 0
while maxdiff > 0.00001:
    numiters+=1
    prevP = p
    p = adj.T*p*d + jProb
    maxdiff = max(abs(prevP-p))
print(numiters)

36


#### plot max diffs

In [10]:
# p = [1/adj.shape[0]] * adj.shape[0] 
# d = 0.9
# jProb = [(1-d)/len(p)] * len(p)
# maxdiffs = []
# for i in range(100):
#     prevP = p
#     p = adj.T*p*d + jProb
#     maxdiffs.append(max(abs(prevP-p)))

In [11]:
# fig, ax = plt.subplots(1)
# ax.set_xlabel("iteration")
# ax.set_ylabel("pagerank max diff")
# ax.set_title("Maximal page rank difference between iterations")
# ax.plot(maxdiffs)

In [12]:
print(sum(p))

0.9999999999999964


In [13]:
output = []
for i in range(len(p)):
    output.append([names[i],p[i]])
output = pd.DataFrame(output, columns = ['name', 'pageRank'])
output = output.sort_values(by=['pageRank'], ascending=False)
output = output.reset_index().drop(columns=['index'])

print(tabulate(output, headers='keys', tablefmt='psql'))
i=1
for r in output.itertuples():
    print(f"{i} {r[1]} with pagerank: {r[2]}")
    i+=1

In [16]:
names = sorted(np.unique(edges))

In [22]:
zipped = np.column_stack((names,p))
pd.DataFrame(zipped[zipped[:,1].argsort()], columns = ["actor", "pagerank"])

,actor,pagerank
0,30.0,2.437621e-07
1,408263.0,2.678747e-07
2,407942.0,2.678903e-07
3,401722.0,2.679013e-07
4,407372.0,2.679170e-07
...,...,...
410231,89.0,1.612539e-03
410232,591.0,2.089912e-03
410233,595.0,2.106158e-03
410234,593.0,2.375671e-03
